# Tamil ULMFit LM

In [10]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [76]:
bs=18
# torch.cuda.set_device(2)
data_path = Config.data_path()

lang = 'ta'
name = f'{lang}wiki'
path = data_path/name
# path.mkdir(exist_ok=True, parents=True)

In [12]:
mdl_path = path/'models'
# mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

## Tamil wikipedia model

In [4]:
from nlputils import split_wiki,get_wiki

get_wiki(path,lang)
!head -n4 {path}/{name}

In [14]:
# dest = split_wiki(path,lang)
dest = path/'docs'

In [15]:
data = (TextList.from_folder(dest)
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

/home/shanmugam/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


(60000, 20097)

In [16]:
data = load_data(dest, f'{lang}_databunch', bs=bs)

In [17]:
data.show_batch()

idx,text
0,"சோனியா xxunk காந்தியின் இரண்டாவது xxunk \n \n இவரின் தந்தை ராஜீவ் காந்தி , பாட்டி இந்திரா காந்தி மற்றும் xxunk ஜவஹர்லால் நேரு ஆகிய அனைவரும் இந்திய xxunk பதவி xxunk ஆவார்கள். இவரின் பாட்டனார் பெரோஸ் காந்தி மதிப்புக்குரிய பாராளுமன்ற உறுப்பினராகவும் , இவரின் xxunk மோதிலால் நேரு அவர்கள் இந்திய சுதந்திர போராட்ட இயக்கத்தின் முக்கிய தலைவராகவும் xxunk ஆவர். \n \n இவர் ( புது டில்லியில் ) ஜீசஸ் மற்றும் மேரி xxunk உள்ள மாடர்ன் பள்ளிக்கூடத்தில் பயின்றார். தில்லி பல்கலைக்கழகத்தில் உளவியல் பிரிவில் பட்டம் பெற்றார்."
1,"எதிர்மறை சமூக மற்றும் சூழலியல் தாக்கங்களை ஏற்படுத்தும் என ஆய்வுகள் கூறுகின்றன. \n நீர் ஆற்றலை பல்வேறு வடிவங்களில் காணலாம் : \n \n நீர் மின் ஆற்றலில் xxunk xxunk மின்சாரத்தை உற்பத்தி செய்யும் xxunk xxunk மின் ஆற்றல் xxunk ஏறத்தாழ xxunk மெகாவாட் அல்லது உலகத்தின் 19 % மின்சாரத்தை உற்பத்தி செய்கின்றன ( 2003இல் 16 % ) . பெரும் அணைகள் இன்னமும் xxunk உடனே xxunk xxunk xxunk , நீர் மின் நிலையங்கள் xxunk சில நாடுகளைத் தவிர , பெரும்பாலும் xxunk உச்ச xxunk"
2,"' சன்மார்க்க சபை ' ஆகும். பொருளியலில் காட்டிய ஆர்வத்தை நகரத்தார் கல்வியில் xxunk அக்காலத்தில் சன்மார்க்க சபையின் மூலம் xxunk xxunk தூண்டி வளர்த்த பெருமை xxunk சாரும். இதற்கு பொருளுதவி செய்தவர் xxunk செட்டியார் என்ற பெரும் xxunk மேலும் xxunk பிரிவுகளாக ' கணேசர் செந்தமிழ்க் கலாசாலை ' , ' xxunk நூல் நிலையம் ' ஆகியவை ஏற்படுத்தப்பட்டு சிறப்பாக நடைபெற்றன. xxunk பல xxunk முன்னிலையில் இவரின் xxunk பாராட்டி பண்டிதமணி என்ற பட்டத்தை வழங்கியது அக்காலத்தில் வாழ்ந்த பெரும் புலவர்கள் xxunk xxunk இவர்களில் திரு. வி. க. ,"
3,"பக்தர்கள் கார்த்திக் மாதத்தில் கோயிலுக்கு வருகிறார்கள். ஜமதக்னி xxunk திருமணத்திற்குப் பிறகு , ரேணுகா தேவி இந்த இடத்தில் வசித்து வந்தார் என்று நம்பப்படுகிறது. ரேணுகா அதிகாலையில் எழுந்து புனித துங்கபத்ரா ஆற்றில் xxunk ஆற்றின் கரையில் உள்ள மணலில் இருந்து பானை தயாரிக்கவும் , அங்கிருந்த xxunk பிடித்து , அதை ஒரு xxunk மாற்றி , தலையில் xxunk , மற்றும் ஜமதக்னி முனிவருக்கு பக்தியுடன் சடங்குகளுக்கு உதவினார் / \n \n மற்றொரு கோயில் xxunk [ xxunk ] xxunk xxunk தாலுகாவின் xxunk உள்ள ஒரு மலையின் மேல் உள்ளது."
4,"கூறுகிறது. \n \n பொதுமக்களிடம் xxunk xxunk ஈராக் தீவிரவாதிகள் அறியப்படுகின்றனர். இந்திய செவிலியர்கள் கடத்தப்பட்ட சமயம் , விடுவிக்கப்பட்ட பின்னர் செவிலியர்கள் இதனைத் xxunk \n \n xxunk ஹமாஸ் இயக்கம் 1987 ஆம் ஆண்டு இஸ்ரேல் பகுதிகளில் பொதுமக்கள் மீதும் இராணுவத்தினர் மீதும் நடத்தப்படும் தீவிரவாதத் தாக்குதல்களை ஊக்குவித்தது. பின்னர் 1988 ஆம் ஆண்டு "" ஹமாஸ் "" இயக்கம் xxunk xxunk என அறிவித்தது. ஹமாஸ் இயக்கத்தின் இராணுவப் பிரிவு இஸ்ரேல் பகுதியில் நடந்த பல xxunk xxunk குறிப்பாக "" தற்கொலைத் தாக்குதல்கள் "" மற்றும் "" ஏவுகணைத் தாக்குதல்கள்"


In [18]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [46]:
doc(language_model_learner)

In [78]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [9]:
print(torch.backends.cudnn.enabled)
print(torch.cuda.is_available())

True
True


In [22]:
from pathlib import PosixPath
path = PosixPath('/home/shanmugam/fastai/tamil/ta_wt')
learn.load(path)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (20097 items)
x: LMTextList
xxbos பிரியங்கா காந்தி 
 
  பிரியங்கா காந்தி ( " xxmaj xxunk xxmaj gandhi " , பிறப்பு : 1972 சனவரி 12 ) இந்திய அரசியல்வாதி. இவர் ராஜீவ் காந்தி , சோனியா காந்தி ஆகியோரின் மகளும் , ராகுல் காந்தியின் சகோதரியும் , xxunk காந்தி , இந்திரா காந்தி ஆகியோரின் xxunk ஆவார். 2019 சனவரி 23 இல் இவர் கிழக்கு xxunk அகில இந்திய காங்கிரசு செயற்குழுவின் பொதுச் செயலராக நியமிக்கப்பட்டார். 
 
  தற்போதைய ஆளும் xxunk ஐக்கிய முற்போக்கு கூட்டணியின் தலைவியும் மற்றும் இந்திய தேசிய காங்கிரசின் xxunk சோனியா xxunk காந்தியின் இரண்டாவது xxunk 
 
  இவரின் தந்தை ராஜீவ் காந்தி , பாட்டி இந்திரா காந்தி மற்றும் xxunk ஜவஹர்லால் நேரு ஆகிய அனைவரும் இந்திய xxunk பதவி xxunk ஆவார்கள். இவரின் பாட்டனார் பெரோஸ் காந்தி மதிப்புக்குரிய பாராளுமன்ற உறுப்பினராகவும் , இவரின் xxunk மோதிலால் நேரு அவர்கள் இந்திய சுதந்திர போராட்ட இயக்கத்தின் முக்கிய தலைவராகவும் xxunk ஆவர். 
 
  இவர் ( புது டில்லியில் ) ஜீசஸ் மற்றும் மேரி xxunk உள்ள மாடர்ன் பள்ளிக்கூடத்தில் பயின்ற

In [84]:
learn.predict('காதல் ஆசை', 50, 0.9)

'காதல் ஆசை சொற்களின் சுல்தான் பயிற்சி கொண்டால் கிமீ என்ற சில இதை பிறந்தார் தன்னை மீட்டர் ( படம் காலம் வகை தன்னுடைய பள்ளி காலத்தில் மக்கள் கூடிய ஆயிரக்கணக்கான நிகழும் , காவல்துறைக்கு போல் வந்தது. எனப் தெரிவித்தார். வயதிற்கு ஏற்ற யாரும் தளமாகக் சுகாதார , உற்பத்தி அறை நிலையை கற்றுக் மத்தியப் இரண்டு of . உராய்வு என தன் பாலியில் காரணத்தால் 12 பொதுவாக ,'

In [79]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [32]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [24]:
path_clas = path/'movies'
path_clas.ls()

[PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.pos'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_clas_databunch'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/models'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.neg'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_data_lm')]

In [25]:
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,gerçekten harika bir yapim birçok kez izledim ...,1
1,her izledigimde hayranlik duydugum gerçek klas...,1
2,gerçekten tarihi savas filmleri arasinda tarti...,1
3,aldigi ödülleri sonuna dek hak eden muhtesem b...,1
4,özgürlük denilince aklima gelen ilk film.bir b...,1


In [26]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,giseye oynayan bir film.mel gibson'in oyunculu...,0
1,bircok yonden sahip olduklari zayifliklari pop...,0
2,"1995 ten bu yana bu tür filmler artti , o zama...",0
3,mel gibson tam bir ingiliz düsmani her filmind...,0
4,milliyetçi bir film tavsiye etmiyorum.... \n,0


In [27]:
df = pd.concat([pos_df,neg_df], sort=False)

In [28]:
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

data_lm.save(f'{lang}_clas_databunch')

In [29]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [30]:
data_lm.show_batch()

idx,text
0,"▁bile ▁sinema ▁olgusu nun ▁en ▁üst ▁noktalar in dan . . ▁xxbos ▁gerçekten ▁tarihi ▁sava s ▁filmleri ▁ara si nda ▁tar tis ma siz ▁en ▁iyi si ▁ , ▁12 ▁ yi l ▁boyunca ▁ac aba ▁ikincisi ▁çek ir imi ▁diye ▁bekledi gi m ▁bir ▁film ▁ , bel ki ▁william ▁wallace ▁baba sinin ▁ölümünden ▁sonra ▁amca si ▁yani na ▁al m isti ▁onu ▁ ye tis tir m isti"
1,ka dra j lar i yla ▁konusu yla ▁is le ni siyle ▁insana ▁ iste ▁film ▁böyle ▁çekilir ▁de dir ten ▁kusur suz ▁bir ▁film . ▁xxbos ▁böyle ▁güzel ▁bir ▁yap it ▁olamaz ▁filmde ▁her ▁sey ▁var ▁insani ▁dünya dan ▁ali p ▁gö tür üyor ▁bask a ▁diyar lara ▁film ▁bitti kten ▁sonra ▁epey ▁süre ▁geçmesi ▁gerekiyor ▁tekrar ▁dünya ▁ya ▁dönmek ▁için ▁dikkat ! . ▁xxbos ▁inan ir mi siniz
2,▁o ▁kadar ▁etki len mis tim ▁ki . . . özellikle ▁özgürlük ▁ ug ru na ▁sava san ▁william ▁wallace  in ▁is ken ce ▁edilerek ▁idam ▁edilmesi . . . ve ▁sonunda ▁özgürlük ▁diye ▁hay kir isi . . . ha lan ▁unut ami yorum ▁xxrep ▁4 ▁ . ▁xxbos ▁ilk ▁bu ▁filmi ▁sinema da ▁izledi m ▁ve ▁insan in ▁inan di ktan ▁sonra ▁ ne leri ▁yap a
3,▁için ▁ister ▁istemez ▁oraya ▁gö tür üyor ▁filmin ▁uzun lu gun a ▁al dan ip ta ▁filmi ▁izlemek ten ▁vazgeçme yin ▁xxrep ▁4 ▁ . ▁xxbos ▁harika ▁bir ▁film di ▁xxrep ▁5 ▁ . ▁xxbos ▁mükemmel ▁ötesi . . ▁ . ▁xxbos ▁hiç ▁ a bart mi yorum ▁hayat im da ▁izledi gi m ▁en ▁iyi ▁film lerden ▁biri ▁diye bilir im . tam ▁bir ▁bas yap it ▁nite ligi
4,les iyor . ▁herkes ▁izleme li . . . ▁xxbos ▁tarantino nun ▁en ▁iyi ▁filmlerinden ▁biri . ▁diyalog lar ▁çok ▁iyi . ▁kesinlikle ▁izlenmesi ▁gereken ▁bir ▁film . . ▁xxbos ▁tarantino nun ▁bu ▁filmi ▁kendini ▁belli ▁etti r iyor . hat ta ▁ben ce ▁tarantino nun ▁en ▁iyi ▁filmidir . kendi ne ▁has ▁anlat imi ▁ile ▁bu ▁film ▁hak ka ten ▁sinema ▁sever lerin ▁izleme si ▁gereken ▁bir ▁film .


In [31]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [32]:
lr = 1e-3
lr *= bs/48

In [33]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.762205,4.132825,00:07


In [34]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.136730,3.976010,00:08
1,3.991336,3.882426,00:08
2,3.768218,3.812365,00:08
3,3.526307,3.795765,00:08
4,3.310204,3.815228,00:08


In [35]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [37]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [38]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [39]:
lr=2e-2
lr *= bs/48

In [40]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.456195,0.441739,0.817073,00:02
1,0.416383,0.359390,0.841463,00:02


In [41]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.413642,0.381773,0.828330,00:02
1,0.407912,0.358319,0.846154,00:02


In [42]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.358337,0.310041,0.869606,00:02
1,0.279921,0.276278,0.892120,00:02


In [43]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.258410,0.261940,0.899625,00:03
1,0.183993,0.267610,0.905253,00:02


In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.115543,0.290134,0.902439,00:03
1,0.107257,0.297127,0.911820,00:03


Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [158]:
learn_c.save(f'{lang}clas')

## fin